In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
# from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.preprocessing import sequence
# import string
from tensorflow.keras.layers import  Embedding,Dense,Conv1D,Input,Flatten,LeakyReLU,Concatenate
from tensorflow.keras.models import Model
import os
import re


# اخبار

In [2]:
news = pd.read_csv("E:\\every_thing_about_python\\rahnamaKalej\\machine learning bootcamp\\DataSets\\MINDlarge_train\\news.tsv"
        ,sep='\t',header=None,)

news.columns  =["NewsID"
        ,"Category","SubCategory","Title","Abstract"
            ,"URL","Title Entities","Abstract Entities"]

# کاربران

In [3]:
users = pd.read_csv("E:\\every_thing_about_python\\rahnamaKalej\\machine learning bootcamp\\DataSets\\MINDlarge_train\\behaviors.tsv"
        ,sep='\t',header=None,)

users.columns = ["impression ID","UserID","Time","History","Impressions"]

# (فیلتر اول) ساخت کلاس برای لایه‌ی اول معماری

In [69]:
class Favorite_Categories():

    def __init__(self , users ,news):

        
        self.news                   = (news.copy(deep=True)).set_index("NewsID")

        self.users                  = users.copy(deep=True)
        self.number_of_categoeies   = len(set(self.news.Category))
        
        self.users.fillna(" ",inplace=True)
        
        

        self.category_of_news       = list(set(self.news.Category))


    def get_favorite_categories_in_term(self,userid):

        #  تاریخچه ی اخبار مشاهده شده توسط کاربر
        
        history                     = " ".join(set(self.users[self.users.UserID==f"U{userid}"].History)).split()
    

        category_history            = [ self.news.loc[news].Category    for news in history]
        category_history            = ' '.join(category_history)
        
        favorite_dict               ={ f"{category}":category_history.count(category) 
                                        for category in self.category_of_news if category in category_history }

        



        if favorite_dict:
            
            Eta                         = sum([v    for k,v in favorite_dict.items()])
            ProbabilityDistribution     = { k:v/Eta   for k,v in favorite_dict.items()}

            #  در این لیست ، دسته ی اخباری که مشاهده شده اند، به ترتیب بیشترین بازدید تا کمترین بازدید چیده می شوند
            favorit_list            =[]

            while favorite_dict:

                max_seen_category   = max( favorite_dict ,key=favorite_dict.get )
                favorite_dict.pop(max_seen_category)

                favorit_list.append(max_seen_category)

            return favorit_list  , ProbabilityDistribution
        
        else:
            ProbabilityDistribution     = { category : (1/len(self.category_of_news))   for category in self.category_of_news}

            return  self.category_of_news  , ProbabilityDistribution
            



        
Filter1 = Favorite_Categories(users ,news)


In [5]:
__userid = 69
print(f"User{__userid} likes this categories: ",", ".join(Filter1.get_favorite_categories_in_term(userid=__userid)[0]))


User69 likes this categories:  lifestyle, tv, entertainment, sports


# RL_RS تعریف محیط برای لایه‌ی دوم معماری 

In [233]:
class SecondFilterEnvironment():
                                    
    def __init__(self,users , news ,max_features_for_Title=5000):
        
        # max_feature_news ==> در یک متن پیدا میکند Tfidf حداکثر تعداد لغاتی که بردار 

        # تمامی کاربران
        self.users                  = users.copy(deep=True)
        # خالی قرار داده میشود  String  یک  NAN برای کاربرانی که تاریخچه ای ندارند، به جای 
        self.users.History.fillna(" ",inplace=True)
        
        self.news                   = (news.copy(deep=True)).set_index("NewsID")
        self.number_of_categoeies   = len(set(self.news.Category))

        #  تمامی کلماتی که برای پردازش نیاز داریم 
        AllCategories               = " ".join(set(self.news.Category))
        AllSubCategories            = " ".join(set(self.news.SubCategory))

        #  اعداد از عناوین اخبار حذف میشوند
        AllWordsFromTitle           = " ".join(self.news.Title)
        AllWordsFromTitle           = re.sub('[0-9]','',AllWordsFromTitle) 


        #    هاSubCategory   ها  و یاCategory به دلیل این که ممکن است بعضی از 
        # استفاده شد  TFIDF تشخیص داده نشود، به همین منظور از سه عدد  TFIDF توسط 
        
        NumFeatures                 = max_features_for_Title - (len((set(self.news.Category))) + len(set(self.news.SubCategory)))
        
        self.NewsToArray_Category   = TfidfVectorizer(token_pattern=r"\S+")
        self.NewsToArray_SubCategory= TfidfVectorizer(token_pattern=r"\S+")
        self.NewsToArray_Title      = TfidfVectorizer(max_features=NumFeatures,stop_words="english")

        self.NewsToArray_Category.fit([AllCategories])
        self.NewsToArray_SubCategory.fit([AllSubCategories])
        self.NewsToArray_Title.fit([AllWordsFromTitle])




    # DO NOT USE THIS METHOD!!!!!!!!!!!!!!!!!!!!!!!!!!!!   
    # this is a protected method that gives state  in vector form and Text form
    def _get_state(self,userid ):

        UserHistory     = " ".join(set(self.users[self.users.UserID==f"U{userid}"].History)).split()
        
        Category        = ""   
        
        SubCategory     = ""  

        Title           = ""
        
        for news in UserHistory:

            Category   += " " + self.news.loc[news].Category
            SubCategory+= " " + self.news.loc[news].SubCategory
            Title      += " " + self.news.loc[news].Title

        # با کمک این بردار میتوان به ویژگی های متون خبری ای که کاربر قبلا مشاهده کرده دست یافت

        CategoryVec       = self.NewsToArray_Category.transform([Category]).toarray()*100
        SubCategoryVec    = self.NewsToArray_SubCategory.transform([SubCategory]).toarray()*100
        TitleVec          = self.NewsToArray_Title.transform([Title]).toarray()*100
        
        StateVec          = np.hstack([np.squeeze(CategoryVec) ,np.squeeze(SubCategoryVec) ,np.squeeze(TitleVec)])[np.newaxis ,:]
        StateText         = [Category , SubCategory , Title]     

        return StateText , StateVec
    
    


    # DO NOT USE THIS METHOD!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # this is a protected method 
    def _get_state_actionReward_newState(self,userid ):
        
        StateText ,StateVector                    = self._get_state(userid)


        #  تمامی اخباری که به کاربر پیشنهاد شده است.
        Impressions                               = " ".join(self.users[self.users.UserID==f"U{userid}"].Impressions).split()

        #  تعریف یک دیکشنری برای این که مشخض شود هر کدام از اخبار پیشنهادی به کاربر، دیده شده است یا خیر
        ImpressionsDict                           = { im.split("-")[0]:int(im.split("-")[-1])     for im in Impressions}
        # /////////////////////
        
        # به روز رسانی گذشته ی کاربر
        # NewStateText                              =  ""
        # NewStateText                              += StateText


        NewStateText_Category                     = StateText[0][::]
        NewStateText_SubCategory                  = StateText[1][::]
        NewStateText_Title                        = StateText[2][::]

    

        LikedNewsText_Category                    = ""
        LikedNewsText_SubCategory                 = ""
        LikedNewsText_Tile                        = ""


        DisLikedNewsText_Category                 = ""
        DisLikedNewsText_SubCategory              = ""
        DisLikedNewsText_Tile                     = ""



        Reward                                    =  19
        PunishMent                                =  -8

        #  به ازای هر خبری که مشاهده شده 19 امتیاز مثبت و به ازای هر خبری که مشاهده نشده 8 امتیاز منفی دریافت میشود
        RecievedRewards                           = {"Punishment":0 ,"Reward":0}

        for news,seen in ImpressionsDict.items():
            
            Category                              = self.news.loc[news].Category
            Subcategory                           = self.news.loc[news].SubCategory
            Title                                 = self.news.loc[news].Title


            #به روز رسانی میشود newstate اگر کاربر خبر را مشاهده کرده باشد، پس بردار مربوط به ویژگی های کاربر به در غالب 
            # اضافه میشود  LikedNews  همچنین کلمات استفاده شده در این خبر برای کشف ویژگی های خبر مورد علاقه ی کاربر به متغیر
            if seen==1:

                NewStateText_Category                 += " " + Category
                NewStateText_SubCategory              += " " + Subcategory
                NewStateText_Title                    += " " + Title


                LikedNewsText_Category                += " " + Category
                LikedNewsText_SubCategory             += " " + Subcategory
                LikedNewsText_Tile                    += " " + Title

                RecievedRewards["Reward"]             += Reward
            
            else:
               

                DisLikedNewsText_Category             += " " + Category
                DisLikedNewsText_SubCategory          += " " + Subcategory
                DisLikedNewsText_Tile                 += " " + Title

                RecievedRewards["Punishment"]         += PunishMent

        # گذشته ی به روز رسانی شده ی کاربر
    
        
        NewStateVector_Category                    = self.NewsToArray_Category.transform([NewStateText_Category]).toarray()*100
        NewStateVector_SubCategory                 = self.NewsToArray_SubCategory.transform([NewStateText_SubCategory]).toarray()*100
        NewStateVector_Title                       = self.NewsToArray_Title.transform([NewStateText_Title]).toarray()*100

        NewStateVector                             = np.hstack([np.squeeze(NewStateVector_Category),np.squeeze(NewStateVector_SubCategory),np.squeeze(NewStateVector_Title)])[np.newaxis ,:]

        # ویژگی های متون خبری ای که کاربر دوست داشته
        
        LikedNewsVector_Category                    = self.NewsToArray_Category.transform([LikedNewsText_Category]).toarray()*100
        LikedNewsVector_SubCategory                 = self.NewsToArray_SubCategory.transform([LikedNewsText_SubCategory]).toarray()*100
        LikedNewsVector_Title                       = self.NewsToArray_Title.transform([LikedNewsText_Tile]).toarray()*100

        LikedNewsVector                             = np.hstack([np.squeeze(LikedNewsVector_Category),np.squeeze(LikedNewsVector_SubCategory),np.squeeze(LikedNewsVector_Title)])[np.newaxis ,:]
       
        # ویژگی های متون خبری ای که کاربر دوست نداشته
        
        DisLikedNewsVector_Category                = self.NewsToArray_Category.transform([DisLikedNewsText_Category]).toarray()*100
        DIsLikedNewsVector_SubCategory             = self.NewsToArray_SubCategory.transform([DisLikedNewsText_SubCategory]).toarray()*100
        DisLikedNewsVector_Title                   = self.NewsToArray_Title.transform([DisLikedNewsText_Tile]).toarray()*100

        DisLikedNewsVector                         = np.hstack([np.squeeze(DisLikedNewsVector_Category),np.squeeze(DIsLikedNewsVector_SubCategory),np.squeeze(DisLikedNewsVector_Title)])[np.newaxis ,:]


        Result                                     ={ "state":StateVector , "LikedNewsVector":LikedNewsVector ,
                                                      "DisLikedNewsVector":DisLikedNewsVector ,"newstate":NewStateVector  }
        
        
        return {**Result , **RecievedRewards}
    

    #  ONLY USE THIS METHOD*****************************
    def get_transitions(self,userid:int)->(dict):

        Transitions =  self._get_state_actionReward_newState(userid)

        return Transitions



env = SecondFilterEnvironment(users , news ,5000)



# del users , news

In [25]:
env.get_transitions(83)["LikedNewsVector"].shape , env.get_transitions(212)["DisLikedNewsVector"].shape ,env.get_transitions(212)["state"].shape,env.get_transitions(212)["newstate"].shape






((1, 5000), (1, 5000), (1, 5000), (1, 5000))

#   ساخت مدل 

In [26]:
def create_model(StateShapeUser,StateShapeNews):
    

    UserFeatures = Input(shape=StateShapeUser,name="User Features")

    NewsFeatures = Input(shape=StateShapeNews,name="News Features")

    user         = Conv1D(128,1,input_shape=(StateShapeUser))(UserFeatures)
    user         = LeakyReLU()(user)
    user         = Flatten()(user)
    user         = Dense(128)(user)
    user         = LeakyReLU()(user)


    news         = Conv1D(128,1,input_shape=(StateShapeNews))(NewsFeatures)
    news         = LeakyReLU()(news)
    news         = Flatten()(news)
    news         = Dense(128)(news)
    news         = LeakyReLU()(news)


    concate      = Concatenate()([user ,news])

    concate      = Dense(256)(concate)
    concate      = LeakyReLU()(concate)

    out          = Dense(2)(concate)




    model        = Model(inputs=[UserFeatures,NewsFeatures],outputs=out) 

    model.summary()

    return model



UserStateShape  =(1,5000)
NewsStateShape  =(1,5000)

model           = create_model( UserStateShape , NewsStateShape )
target_model    = create_model( UserStateShape , NewsStateShape )

target_model.set_weights(model.get_weights())

#//////////////  loss_function( y_true , y_predict )
loss_function = tf.keras.losses.Huber()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 User Features (InputLayer)     [(None, 1, 5000)]    0           []                               
                                                                                                  
 News Features (InputLayer)     [(None, 1, 5000)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1, 128)       640128      ['User Features[0][0]']          
                                                                                                  
 conv1d_1 (Conv1D)              (None, 1, 128)       640128      ['News Features[0][0]']          
                                                                                              

#  ساخت یک کلاس برای آموزش راحت تر مدل

In [27]:

class Dqn_model():

    def __init__(self,env,model ,target_model ,loss_function ,optimizer ,discount_factor ,action_size= 2):

        self.env            = env
        self.action_size    = action_size
        self.model          = model
        self.target_model   = target_model
        self.loss_function  = loss_function
        self.optimizer      = optimizer
        self.discount       = discount_factor
        
    # private method
    #   this method is the core to learn policy
    @tf.function
    def __train(self,StateUser,StateNews ,action ,reward ,NewStateUser):

        with tf.GradientTape() as tape:

            Q1 = self.model([StateUser ,StateNews] ,training=True)
            Q2 = Q1[::]

            reward = tf.cast(reward ,tf.float32)
            Y = reward + self.discount * tf.reduce_max( self.target_model([NewStateUser,StateNews] , training=True) ,axis=-1 )
            
            
            action_index = tf.one_hot([action] , self.action_size)

            Q2 = tf.tensor_scatter_nd_update(Q2 , tf.where(action_index)  , Y)
            
            loss = self.loss_function(Q2 ,Q1)

        grad = tape.gradient(loss , self.model.trainable_variables)
        self.optimizer.apply_gradients(zip( grad , self.model.trainable_variables))
        
        return loss

    # private method
    #  This method helps to put the above method in a for loop
    def __learn(self,StateUser,StateNews ,action ,reward ,NewStateUser ):
        
        loss = self.__train(StateUser,StateNews ,action ,reward ,NewStateUser)

        return loss
        
    # به روز رسانی شبکه هدف
    @tf.function
    def soft_Update(self,  target_model_weights    ,  model_weights   ,  tau=0.01):

        for TM , M in zip(target_model_weights ,model_weights):
            TM.assign(  (1-tau)*TM  +  tau * M   )
        

    #  این تابع ، با استفاده از  محیط، به شبکه ی عصبی، ویژگی های اخبار مورد علاقه ی و  کاربر مورد نظر را آموزش می دهد
    def Learn_favorite_category(self,userid , n_epocks=1 ,early_stopping = 0.01):

        transition          = self.env.get_transitions(userid) 

        StateUser           = transition["state"][np.newaxis,:,:]
        NewStateUser        = transition["newstate"][np.newaxis,:,:]
        LikedNewsVector     = transition["LikedNewsVector"][np.newaxis,:,:]
        DisLikedNewsVector  = transition["DisLikedNewsVector"][np.newaxis,:,:]
        ActionForLike       = 1
        ActionForDislike    = 0

        Reward              = transition["Reward"]
        Punishment          = transition["Punishment"]

        TotalLoss=[]
        for epock in range(n_epocks):
            
            loss1          = self.__learn(StateUser,LikedNewsVector    ,ActionForLike    ,Reward     ,NewStateUser )
            loss2          = self.__learn(StateUser,DisLikedNewsVector ,ActionForDislike ,Punishment ,NewStateUser ,)
            
            TotalLoss.append(loss1+loss2)
            
            if (loss1 + loss2)<=early_stopping:
                break        
                
        MeanTotalLoss     = np.mean(TotalLoss)

        return MeanTotalLoss

        





dqn                     = Dqn_model(env ,model ,target_model ,loss_function ,optimizer,0.9)




#  debugging for Env and DQN classes......

In [28]:
for i in np.random.randint(low =0 ,high = 711221,size= 10):
    
    #می‌باشد  __train   در بالای تابع   @tf.function    مشاهده شد، مربوط به    warning  اگر در این جا 
    print(f"dqn is learning User{i}")
    dqn.Learn_favorite_category(i)

dqn is learning User592649
dqn is learning User454964
dqn is learning User278140
dqn is learning User48575
dqn is learning User340821
dqn is learning User257367
dqn is learning User703341


# train DQN

In [32]:
n_episodes              = 10   #     تعداد کل مراحل آموزشی 
length_of_episode       = 10   #    در هر بار آموزش شبکه عصبی، چند کاربر به شبکه عصبی آموزش داده میشود

n_iterations            = 10   #     علایق هر کاربر، چند مرتبه به شبکه عصبی آموزش داده می‌شود

number_of_users         = 711221     #   تعداد کل  کاربران موجود در دیتاست


np.random.choice(range(number_of_users), length_of_episode)
# range(start_index , end_index )
total_loss              = []

for episode in range(n_episodes):

   print(f"Episode {episode+1} in progress.....")

   #یاد گرفته شود    end_index    تا کاربر شماره ی     start_index      از کاربر شماره ی    
   # start_index          = np.random.randint( low=0   ,  high= number_of_users -length_of_episode)
   # end_index            = start_index + length_of_episode + 1
   
   episode_loss         = []
   for userid in np.random.choice(range(number_of_users), length_of_episode):
      
      loss_ = dqn.Learn_favorite_category(userid , n_iterations )

      episode_loss.append(loss_)

   episode_loss         = np.mean(episode_loss)
   total_loss.append(episode_loss)

      
   if episode%10        ==0:
      
      dqn.soft_Update(dqn.target_model.trainable_variables,dqn.model.trainable_variables )
   
  


   print(f"episode {episode +1} finished. loss = {episode_loss} \n","="*50)





Episode 1 in progress.....
episode 1 finished. loss = 28.54543685913086 
Episode 2 in progress.....
episode 2 finished. loss = 31.748987197875977 
Episode 3 in progress.....
episode 3 finished. loss = 17.15983009338379 
Episode 4 in progress.....
episode 4 finished. loss = 33.222049713134766 
Episode 5 in progress.....
episode 5 finished. loss = 11.866804122924805 
Episode 6 in progress.....
episode 6 finished. loss = 30.54372215270996 
Episode 7 in progress.....
episode 7 finished. loss = 49.46721649169922 
Episode 8 in progress.....
episode 8 finished. loss = 12.67637825012207 
Episode 9 in progress.....
episode 9 finished. loss = 112.05821990966797 
Episode 10 in progress.....
episode 10 finished. loss = 191.76950073242188 


# ساخت کلاس سیستم توصیه گر

In [280]:
class RecommenderSystem(SecondFilterEnvironment):

    def __init__(self , Filter1:Favorite_Categories , Filter2:Model ,Users:pd.DataFrame, News:pd.DataFrame  )->None:

        super().__init__(Users , News)
        self.Filter1                    = Filter1
        self.Filter2                    = Filter2

        self.News                       =  News


    # this is a private method
    # Do not use this method outside of class!!!!!!!!!!!
    def __GetNewsFeatures(self, NewsID)->np.array:

        Category_Text                   =  self.news.loc[NewsID].Category
        SubCategory_Text                =  self.news.loc[NewsID].SubCategory
        Title_Text                      =  self.news.loc[NewsID].Title

        Category_Vec                    =  self.NewsToArray_Category.transform([Category_Text]).toarray()
        SubCategory_Vec                 =  self.NewsToArray_SubCategory.transform([SubCategory_Text]).toarray()
        Title_Vec                       =  self.NewsToArray_Title.transform([Title_Text]).toarray()

        NewsFeatures                    =  np.hstack([np.squeeze(Category_Vec),np.squeeze(SubCategory_Vec),np.squeeze(Title_Vec)])[np.newaxis,:]

        return NewsFeatures

    # this is a private method
    # Do not use this method outside of class!!!!!!!!!!!
    def __GetSeenedNews(self , UserID:int) ->set:
        

        Set_history                     = set(" ".join(set(env.users[env.users.UserID==f"U{UserID}"].History)).split())

        Set_Impressions                 = set([news.split("-")[0]   for news  in " ".join(env.users[env.users["UserID"]==f"U{UserID}"]["Impressions"]).split()])
        
        Union                           = Set_history   |  Set_Impressions
        
        return Union



    

    #   Main method
    def OfferNews(self , UserID , MaxNumberOfNews=10) ->list:

        FavoriteCategories              = self.Filter1.get_favorite_categories_in_term(UserID)[-1]

        sortedDict_FavoriteCategories   =  sorted(FavoriteCategories.items() , key = lambda X :X[-1] , reverse=True)

        List_FavoriteCategories         = list(map(lambda X :X[ 0] ,sortedDict_FavoriteCategories))
        List_Distribution               = list(map(lambda X :X[-1] ,sortedDict_FavoriteCategories))


        UserFeatures                    = self._get_state(UserID)[-1]

        #   با توجه به آمار کتگوری‌های دیده شده، یک لیست ایجاد میشود تا از دسته‌های محبوب، اخبار بیشتری پیشنهاد گردد
        SampleCategories                = [np.random.choice(List_FavoriteCategories, p=List_Distribution)    for _ in range(MaxNumberOfNews)   ] 

        #  اخباری که قرار است پیشنهاد شوند
        OfferedNews                     = []


        AlreadySeenedNews               = self.__GetSeenedNews(UserID)


        while len(OfferedNews)< MaxNumberOfNews:
            
        
            SelectedCategory           =   SampleCategories.pop(0)

            while True:
                
                NewsID                 = ((self.News[self.News["Category"]==SelectedCategory]).sample(1))["NewsID"].to_list()[0]

                NewsFeatures           =  self.__GetNewsFeatures(NewsID)  

                #   اگر آن خبر مورد نظر، خبری بود که کاربر مشاهده نکرده است
                if NewsID not in (set(OfferedNews)  |  AlreadySeenedNews):

                    if (np.argmax(model([UserFeatures[np.newaxis,:] , NewsFeatures[np.newaxis,:]  ]))) == 1:

                        OfferedNews.append(NewsID)
                        
                        break
            

          
                

        return OfferedNews 
        
    
            

            

        


recommender = RecommenderSystem(Filter1 , model ,users,news )
    

# نمونه خروجی سیستم توصیه گر پس از اتصال تمامی فیلتر‌ها

In [310]:
userid__    = 15

R_News      =recommender.OfferNews(UserID=userid__,MaxNumberOfNews=10)


print(f"recommended news to User {userid__}: " + "{" + ",  ".join(R_News) + "}" )


recommended news to User 15: {N56588,  N21346,  N6035,  N5438,  N89302,  N47810,  N6982,  N75484,  N105411,  N110420}
